In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_lg

!pip install typer
#!pip install spacy[cuda110]

     |████████████████████████████████| 5.8 MB 12.6 MB/s 
     |████████████████████████████████| 10.1 MB 47.7 MB/s 
     |████████████████████████████████| 623 kB 26.2 MB/s 
     |████████████████████████████████| 456 kB 47.9 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 777.1 MB 16 kB/s 
✔ Download and installation successful
You ca

In [2]:
import spacy
import srsly
import pandas as pd
spacy.prefer_gpu()

True

In [3]:
# download training set
!wget https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_drugs/assets/drugs_training.jsonl

# download dev set
!wget https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_drugs/assets/drugs_eval.jsonl

# download script converting jsonl to .spacy format
!wget https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_drugs/scripts/preprocess.py

--2021-09-19 03:11:35--  https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_drugs/assets/drugs_training.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3502481 (3.3M) [text/plain]
Saving to: ‘drugs_training.jsonl’

drugs_training.json 100%[===================>]   3.34M  --.-KB/s    in 0.03s   

2021-09-19 03:11:36 (97.5 MB/s) - ‘drugs_training.jsonl’ saved [3502481/3502481]

--2021-09-19 03:11:36--  https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_drugs/assets/drugs_eval.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request se

#### Convert JSONL to .spaCy format

In [6]:
!python preprocess.py ./drugs_training.jsonl ./drugs_training.spacy
!python preprocess.py ./drugs_eval.jsonl ./drugs_eval.spacy

Processed 1477 documents: drugs_training.spacy
Processed 500 documents: drugs_eval.spacy


#### Generate config

In [7]:
!python -m spacy init config --lang en --pipeline ner configs/config.cfg --force --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
configs/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


#### Training

In [8]:
!python -m spacy train configs/config.cfg --output models/ --paths.train ./drugs_training.spacy --paths.dev ./drugs_eval.spacy --gpu-id 0 --components.tok2vec.model.encode.width 64 --components.tok2vec.model.encode.depth 4

✔ Created output directory: models
ℹ Saving to output directory: models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-09-19 03:16:53,617] [INFO] Set up nlp object from config
[2021-09-19 03:16:53,630] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-09-19 03:16:53,635] [INFO] Created vocabulary
[2021-09-19 03:18:00,793] [INFO] Added vectors: en_core_web_lg
tcmalloc: large alloc 1643593728 bytes == 0x55eb4a536000 @  0x7ff2731552a4 0x7ff2616e8cb0 0x7ff2616f2083 0x7ff2616f187b 0x7ff2616eede1 0x7ff2616ef6de 0x55ea98f042ed 0x55ea98ff5e1d 0x55ea98f77e99 0x55ea98f729ee 0x55ea98f05bda 0x55ea98f74737 0x7ff127f52918 0x7ff127f541fe 0x7ff127f59947 0x7ff127f5ad42 0x55ea98f04c52 0x55ea98f77c25 0x55ea98f05afa 0x55ea98f73915 0x7ff127f52918 0x7ff127f541fe 0x7ff127f59470 0x55ea98f044b0 0x55ea98ff5e1d 0x55ea98f77e99 0x55ea98f72ced 0x55ea98f05bda 0x55ea98f74737 0x55ea98f72ced 0x55ea98f05bda
tcmalloc: large alloc 1662509056 bytes == 0x55eb4a536000 @  0x7ff2731

In [9]:
!python -m spacy evaluate ./models/model-last/ ./drugs_eval.spacy --gpu-id -1 --output evaluate.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     99.99
NER P   78.65
NER R   77.35
NER F   77.99
SPEED   34418


=============================== NER (per type) ===============================

           P       R       F
DRUG   78.65   77.35   77.99

✔ Saved results to evaluate.json


#### Retrain a new model focusing on Recall

In [12]:
!python -m spacy train configs/config.cfg --output models_recall/ --paths.train ./drugs_training.spacy --paths.dev ./drugs_eval.spacy --gpu-id 0 --components.tok2vec.model.encode.width 64 --components.tok2vec.model.encode.depth 4 --training.score_weights.ents_r 1.0 --training.score_weights.ents_f 0.0

ℹ Saving to output directory: models_recall
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-09-19 03:38:19,151] [INFO] Set up nlp object from config
[2021-09-19 03:38:19,164] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-09-19 03:38:19,169] [INFO] Created vocabulary
[2021-09-19 03:38:46,218] [INFO] Added vectors: en_core_web_lg
tcmalloc: large alloc 1643593728 bytes == 0x5620c314a000 @  0x7ff78559d2a4 0x7ff773b30cb0 0x7ff773b3a083 0x7ff773b3987b 0x7ff773b36de1 0x7ff773b376de 0x562043e7c2ed 0x562043f6de1d 0x562043eefe99 0x562043eea9ee 0x562043e7dbda 0x562043eec737 0x7ff63a397918 0x7ff63a3991fe 0x7ff63a39e947 0x7ff63a39fd42 0x562043e7cc52 0x562043eefc25 0x562043e7dafa 0x562043eeb915 0x7ff63a397918 0x7ff63a3991fe 0x7ff63a39e470 0x562043e7c4b0 0x562043f6de1d 0x562043eefe99 0x562043eeaced 0x562043e7dbda 0x562043eec737 0x562043eeaced 0x562043e7dbda
tcmalloc: large alloc 1662509056 bytes == 0x5620c314a000 @  0x7ff78559d2a4 0x7ff773b30cb0 0x7ff77

In [13]:
!python -m spacy evaluate ./models_recall/model-last/ ./drugs_eval.spacy --gpu-id -1 --output evaluate.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     99.99
NER P   77.09
NER R   79.01
NER F   78.04
SPEED   33034


=============================== NER (per type) ===============================

           P       R       F
DRUG   77.09   79.01   78.04

✔ Saved results to evaluate.json


In [14]:
nlp = spacy.load("./models/model-last/")

In [16]:
data = srsly.read_jsonl("./drugs_eval.jsonl")

In [24]:
for idx in range(25):
    test_case = next(data)
    sentence = test_case["text"] 
    doc = nlp(sentence)
    y_pred = {ent: ent.text for ent in doc.ents}
    print(f"\n\n[Sentence]: \n\t{sentence}")
    print(f"\n[Predicted]: \n\t {y_pred}")
    print("-"*100)




[Sentence]: 
	agreed!

[Predicted]: 
	 {}
----------------------------------------------------------------------------------------------------


[Sentence]: 
	Fuck man - shit is dry as balls where I am

[Predicted]: 
	 {}
----------------------------------------------------------------------------------------------------


[Sentence]: 
	I believe with extensions it comes out to 120 days. More then likely you fell through the crack. As the lawyer above posted don't stir the pot.

[Predicted]: 
	 {crack: 'crack', pot: 'pot'}
----------------------------------------------------------------------------------------------------


[Sentence]: 
	just use it with your dboy. by time he figures out its fake, if he even does... he won't know who gave it to him, so fuck it.

[Predicted]: 
	 {}
----------------------------------------------------------------------------------------------------


[Sentence]: 
	I'm paying about £18 for 200mg in the UK (darkweb). No idea what the purity is but it wil